In [294]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
from sklearn.metrics import r2_score
from io import StringIO
import glob
import os
import raytracing as rt
from scipy.ndimage import gaussian_filter

import scipy.optimize as spo
import scipy.constants as spc
import scipy.integrate as spi
import scipy.linalg as spl
import scipy.signal as spsi
import scipy.stats as spst

In [295]:
import pandas as pd

def lire_csv(chemin_fichier):
    # Lire le contenu du fichier
    with open(chemin_fichier, 'r', encoding='utf-8') as fichier:
        contenu = fichier.readlines()

    # Prétraiter les lignes pour remplacer la virgule décimale par un point
    # et la seconde virgule (délimiteur) par un point-virgule
    contenu_traite = []
    for ligne in contenu:
        if ligne != contenu[0]:
            # Remplacer la première virgule (décimale) par un point dans chaque partie numérique
            partie_avant, partie_apres = ligne.split(',', 1)
            partie_avant = partie_avant.replace(',', '.')
            # Ajouter la ligne traitée à la liste
            contenu_traite.append(partie_avant + partie_apres)
        else :
            contenu_traite.append(ligne)
        

    # Convertir le contenu traité en un seul texte
    contenu_traite_str = ''.join(contenu_traite)

    # Utiliser pandas pour lire la chaîne traitée comme un fichier CSV
    from io import StringIO
    df = pd.read_csv(StringIO(contenu_traite_str), delimiter=',')

    return df

In [296]:
def lire_csv(chemin_fichier):
    # Lire le fichier en ignorant la première ligne
    with open(chemin_fichier, 'r', encoding='utf-8') as fichier:
        lignes = fichier.readlines()[1:]  # Ignorer la première ligne
    
    # Préparer le contenu transformé
    contenu_transforme = []
    for ligne in lignes:
        partie_gauche, _, partie_droite = ligne.partition(', ')  # Séparer à la première virgule
        partie_gauche, partie_droite = partie_gauche.replace(',', '.'), partie_droite.replace(',', '.')   # Au cas où, mais devrait être inutile ici
        ligne_transformee = partie_gauche + ';' + partie_droite
        contenu_transforme.append(ligne_transformee)
    
    # Convertir le contenu transformé en un objet StringIO
    contenu_csv = StringIO(''.join(contenu_transforme))
    
    # Lire le contenu transformé en DataFrame
    df = pd.read_csv(contenu_csv, names=['Pixels', 'Value'],sep=";")
    return df

## Données de la partie 1

In [297]:
df_partie_1 = pd.read_excel("Projet_final.xlsx", sheet_name="Partie 1")
print(df_partie_1)

   Volume solution ajoutée (ml)    2D \mu  3D \mu  3D \sigma  2D ln(I_0/I)  \
0                             0  0.000257  0.0004     0.0006        0.0018   
1                             1  0.102100  0.0865     0.0023        0.7147   
2                             2  0.189643  0.1582     0.0030        1.3275   
3                             3  0.269200  0.2167     0.0046        1.8844   
4                             5  0.366271  0.2707     0.0087        2.5639   
5                             7  0.415643  0.2792     0.0122        2.9095   
6                             9  0.438200  0.2737     0.0145        3.0674   
7                            11  0.449114  0.2678     0.0159        3.1438   
8                            13  0.454529  0.2618     0.0167        3.1817   
9                            15  0.458900  0.2581     0.0173        3.2123   

   2D \sigma \ln(I_0/I)  Unnamed: 6  Unnamed: 7  
0                0.0021         NaN         NaN  
1                0.0069         NaN      

In [298]:
Volume = [0,1,2,3,5,7,9,11,13,15]
mu_2d = [0.0003,0.1021,0.1896,0.2692,0.3663,0.4156,0.4382,0.4491,0.4545,0.4589]
mu_3d = [0.0004,0.0865,0.1582,2167,0.2707,0.2792,0.2737,0.2678]

## Données de la partie 2

In [299]:
df_partie_2 = pd.read_excel("Projet_final.xlsx", sheet_name="Partie 2")
print(df_partie_2)

  Unnamed: 0 Ouverture de la fente doigt 1 (plus gros) Unnamed: 3 doigt 2  \
0        NaN                   NaN                  mu      sigma      mu   
1        NaN        pas de fenêtre              0.8061     0.0248  0.8142   
2      ± 0,2                   1.5              0.9016     0.0191  0.9145   
3      ± 0,1                     1              0.9842     0.0214  1.0042   
4      ± 0,1                   0.5                1.11     0.0254  1.1544   

  Unnamed: 5 doigt 3 Unnamed: 7 doigt 4 (plus petit) Unnamed: 9  
0      sigma      mu      sigma                   mu      sigma  
1     0.0097     0.8     0.0082               0.8244     0.0065  
2      0.011  0.9017     0.0132               0.9316     0.0085  
3     0.0151  0.9976     0.0133               0.9932     0.0622  
4     0.0247  1.1639     0.0232               1.2466     0.0257  


## Données de la partie 3

In [300]:
df_partie_3a = pd.read_excel("Projet_final.xlsx", sheet_name="Partie 3a")
df_partie_3b = pd.read_excel("Projet_final.xlsx", sheet_name="Partie 3b")
print(df_partie_3a)
print(df_partie_3b)

       ms  shutter speed (fps)  mu (ln(I_0/I))   sigma
0   79.57                12.53          0.0000  0.0000
1   67.22                14.83          0.0581  0.0028
2   62.91                15.96          0.1166  0.0038
3   57.35                17.39          0.2008  0.0034
4   51.98                19.18          0.2910  0.0040
5   46.92                21.22          0.3883  0.0040
6   40.55                24.56          0.5261  0.0043
7   37.42                26.60          0.6022  0.0045
8   33.86                29.41          0.6985  0.0044
9    1.68                30.43          2.8996  0.0124
10   5.24                30.43          2.2646  0.0067
11   8.15                30.43          1.9427  0.0061
12  13.65                30.43          1.5136  0.0054
13  18.51                30.43          1.2503  0.0052
14  24.32                30.43          1.0029  0.0046
   Unnamed: 0  bruit (%)    mu_B  sigma_B Cône clair  Unnamed: 5  Unnamed: 6  \
0         NaN        NaN     NaN      Na

## Données de la partie 4 

In [301]:
partie4_2d_x = lire_csv(r"partie_4\partie4_2d_x.csv")
partie4_2d_y = lire_csv(r"partie_4\partie4_2d_y.csv")
partie4_3d_040_025_ramlack_x = lire_csv(r"partie_4/partie4_3d_040_025_ramlack_x.csv")
partie4_3d_040_025_ramlack_x_v2 = lire_csv(r"partie_4\partie4_3d_040_025_ramlack_x_v2.csv")
partie4_3d_040_025_ramlack_y_v2 = lire_csv(r"partie_4\partie4_3d_040_025_ramlack_y_v2.csv")
partie4_3d_040_025_ramlack_y = lire_csv(r"partie_4\partie4_3d_040_025_ramlack_y.csv")
partie4_3d_160_025_ramlack_x = lire_csv(r"partie_4\partie4_3d_160_025_ramlack_x.csv")
partie4_3d_160_025_ramlack_y = lire_csv(r"partie_4\partie4_3d_160_025_ramlack_y.csv")
partie4_3d_320_025_dia050_y = lire_csv(r"partie_4\partie4_3d_320_025_dia050_y.csv")
partie4_3d_320_025_dia150_y = lire_csv(r"partie_4\partie4_3d_320_025_dia150_y.csv")
partie4_3d_320_025_nofilter_y = lire_csv(r"partie_4\partie4_3d_320_025_nofilter_y.csv")
partie4_3d_320_025_x = lire_csv(r"partie_4\partie4_3d_320_025_x.csv")
partie4_3d_320_025_y = lire_csv(r"partie_4\partie4_3d_320_025_y.csv")
partie4_3d_320_050_x = lire_csv(r"partie_4\partie4_3d_320_050_x.csv")
partie4_3d_320_050_y = lire_csv(r"partie_4\partie4_3d_320_050_y.csv")
partie4_3d_320_200_x = lire_csv(r"partie_4\partie4_3d_320_200_x.csv")
partie4_3d_320_200_y = lire_csv(r"partie_4\partie4_3d_320_200_y.csv")

## Données de la partie 5

In [302]:
p5_2d_320_025_bas = lire_csv(r"partie 5\2d_320_025_bas.csv")
p5_2d_320_025_haut = lire_csv(r"partie 5\2d_320_025_haut.csv")
p5_2d_320_025_milieu = lire_csv(r"partie 5\2d_320_025_milieu.csv")
p5_3d_040_025_bas = lire_csv(r"partie 5\3d_040_025_bas.csv")
p5_3d_040_025_haut = lire_csv(r"partie 5\3d_040_025_haut.csv")
p5_3d_040_025_milieu = lire_csv(r"partie 5\3d_040_025_milieu.csv")
p5_3d_160_025_bas = lire_csv(r"partie 5\3d_160_025_bas.csv")
p5_3d_160_025_haut = lire_csv(r"partie 5\3d_160_025_haut.csv")
p5_3d_160_025_milieu = lire_csv(r"partie 5\3d_160_025_milieu.csv")
p5_3d_320_025_bas = lire_csv(r"partie 5\3d_320_025_bas.csv")
p5_3d_320_025_haut = lire_csv(r"partie 5\3d_320_025_haut.csv")
p5_3d_320_025_milieu = lire_csv(r"partie 5\3d_320_025_milieu.csv")
p5_3d_320_025_dia050 = lire_csv(r"partie 5\3d_320_025_dia050.csv")
p5_3d_320_025_dia100 = lire_csv(r"partie 5\3d_320_025_dia100.csv")
p5_3d_320_025_dia150 = lire_csv(r"partie 5\3d_320_025_dia150.csv")
p5_3d_320_050_bas = lire_csv(r"partie 5\3d_320_050_bas.csv")
p5_3d_320_050_haut = lire_csv(r"partie 5\3d_320_050_haut.csv")
p5_3d_320_050_milieu = lire_csv(r"partie 5\3d_320_050_milieu.csv")
p5_3d_320_200_bas = lire_csv(r"partie 5\3d_320_200_bas.csv")
p5_3d_320_200_haut = lire_csv(r"partie 5\3d_320_200_haut.csv")
p5_3d_320_200_milieu = lire_csv(r"partie 5\3d_320_200_milieu.csv")